<a href="https://colab.research.google.com/github/arushi-lu/deep_learning/blob/main/AlexNet_c_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        logits = self.classifier(x)
        return logits

In [25]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [29]:
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

[1, 200] loss: 1.081
[1, 400] loss: 1.066
[1, 600] loss: 1.049
[2, 200] loss: 1.041
[2, 400] loss: 1.034
[2, 600] loss: 1.029
[3, 200] loss: 0.997
[3, 400] loss: 0.975
[3, 600] loss: 0.979
[4, 200] loss: 0.949
[4, 400] loss: 0.948
[4, 600] loss: 0.925
[5, 200] loss: 0.916
[5, 400] loss: 0.896
[5, 600] loss: 0.897
[6, 200] loss: 0.872
[6, 400] loss: 0.882
[6, 600] loss: 0.847
[7, 200] loss: 0.818
[7, 400] loss: 0.820
[7, 600] loss: 0.814
[8, 200] loss: 0.777
[8, 400] loss: 0.803
[8, 600] loss: 0.778
[9, 200] loss: 0.755
[9, 400] loss: 0.768
[9, 600] loss: 0.752
[10, 200] loss: 0.743
[10, 400] loss: 0.737
[10, 600] loss: 0.718
[11, 200] loss: 0.703
[11, 400] loss: 0.679
[11, 600] loss: 0.716
[12, 200] loss: 0.683
[12, 400] loss: 0.674
[12, 600] loss: 0.677
[13, 200] loss: 0.655
[13, 400] loss: 0.653
[13, 600] loss: 0.643
[14, 200] loss: 0.617
[14, 400] loss: 0.618
[14, 600] loss: 0.631
[15, 200] loss: 0.597
[15, 400] loss: 0.608
[15, 600] loss: 0.609
[16, 200] loss: 0.592
[16, 400] loss: